# Optimizing Employee Performance With End-to-End HR Analytics Data Pipeline
___

## Introduction
Name  : Yonathan Anggraiwan

This program was designed to implement an end-to-end data pipeline from PostgreSQL to ElasticSearch, utilizing an employee performance and productivity dataset sourced from [Kaggle](https://www.kaggle.com/datasets/mexwell/employee-performance-and-productivity-data). 

The project also includes data validation using the Great Expectations framework, and the results are presented through an interactive dashboard built with Kibana.
___

## Loading Package

In [1]:
import pandas as pd
from great_expectations.data_context import FileDataContext

___
## Loading Clean Data

The dataset used is an employee performance and productivity dataset sourced from [Kaggle](https://www.kaggle.com/datasets/mexwell/employee-performance-and-productivity-data). I will be conducting further exploratory analysis on this dataset to uncover deeper insights.

In [ ]:
df = pd.read_csv("dags/data_clean.csv")
df

,employee_id,department,gender,age,job_title,hire_date,years_at_company,education_level,performance_score,monthly_salary,...,projects_handled,overtime_hours,sick_days,remote_work_frequency,team_size,training_hours,promotions,employee_satisfaction_score,resigned,age_cap
0,1,IT,Male,55,Specialist,2022-01-19,2,High School,5,6750.0,...,32,22,2,0,14,66,0,2.63,False,Middle-Aged
1,2,Finance,Male,29,Developer,2024-04-18,0,High School,5,7500.0,...,34,13,14,100,12,61,2,1.72,False,Young
2,3,Finance,Male,55,Specialist,2015-10-26,8,High School,3,5850.0,...,27,6,3,50,10,1,0,3.17,False,Middle-Aged
3,4,Customer Support,Female,48,Analyst,2016-10-22,7,Bachelor,2,4800.0,...,10,28,12,100,10,0,1,1.86,False,Middle-Aged
4,5,Engineering,Female,36,Analyst,2021-07-23,3,Bachelor,2,4800.0,...,11,29,13,100,15,9,1,1.25,False,Adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,Finance,Male,27,Technician,2022-12-07,1,Bachelor,4,4900.0,...,46,5,3,75,16,48,2,1.28,False,Young
99996,99997,IT,Female,36,Consultant,2018-07-24,6,Master,5,8250.0,...,35,7,0,0,10,77,1,3.48,True,Adult
99997,99998,Operations,Male,53,Analyst,2015-11-24,8,High School,2,4800.0,...,13,6,5,0,5,87,1,2.60,False,Middle-Aged
99998,99999,HR,Female,22,Consultant,2015-08-03,9,High School,5,8250.0,...,43,10,1,75,2,31,1,3.10,False,Young


In [3]:
df.describe()

,employee_id,age,years_at_company,performance_score,monthly_salary,work_hours_per_week,projects_handled,overtime_hours,sick_days,remote_work_frequency,team_size,training_hours,promotions,employee_satisfaction_score
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,50000.500000,41.029410,4.476070,2.995430,6403.211000,44.956950,24.431170,14.514930,7.008550,50.090500,10.013560,49.506060,0.999720,2.999088
std,28867.657797,11.244121,2.869336,1.414726,1372.508717,8.942003,14.469584,8.664026,4.331591,35.351157,5.495405,28.890383,0.815872,1.150719
min,1.000000,22.000000,0.000000,1.000000,3850.000000,30.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
25%,25000.750000,31.000000,2.000000,2.000000,5250.000000,37.000000,12.000000,7.000000,3.000000,25.000000,5.000000,25.000000,0.000000,2.010000
50%,50000.500000,41.000000,4.000000,3.000000,6500.000000,45.000000,24.000000,15.000000,7.000000,50.000000,10.000000,49.000000,1.000000,3.000000
75%,75000.250000,51.000000,7.000000,4.000000,7500.000000,53.000000,37.000000,22.000000,11.000000,75.000000,15.000000,75.000000,2.000000,3.990000
max,100000.000000,60.000000,10.000000,5.000000,9000.000000,60.000000,49.000000,29.000000,14.000000,100.000000,19.000000,99.000000,2.000000,5.000000


___
## Great Expectations Section

In Great Expectations (GE), the first step is to establish the data context by defining the dataset being processed, assigning it a clear name, and configuring its storage location. These preparations ensure that the data is ready for validation using the Great Expectations validator.

In [4]:
# Create a data context
context = FileDataContext.create(project_root_dir='./')

In [6]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'csv-data-ms3-1.3'
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'ms3'
path_to_data = r'D:\1. Data Science\1. Bootcamp\Fase 2\hck028\p2-ftds028-hck-m3-yonathanggraiwan\dags\P2M3_Yonathan_Anggraiwan_data_clean.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

In [7]:
# Creat an expectation suite
expectation_suite_name = 'expectation-data-ms3'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,employee_id,department,gender,age,job_title,hire_date,years_at_company,education_level,performance_score,monthly_salary,...,projects_handled,overtime_hours,sick_days,remote_work_frequency,team_size,training_hours,promotions,employee_satisfaction_score,resigned,age_cap
0,1,IT,Male,55,Specialist,2022-01-19,2,High School,5,6750.0,...,32,22,2,0,14,66,0,2.63,False,Middle-Aged
1,2,Finance,Male,29,Developer,2024-04-18,0,High School,5,7500.0,...,34,13,14,100,12,61,2,1.72,False,Young
2,3,Finance,Male,55,Specialist,2015-10-26,8,High School,3,5850.0,...,27,6,3,50,10,1,0,3.17,False,Middle-Aged
3,4,Customer Support,Female,48,Analyst,2016-10-22,7,Bachelor,2,4800.0,...,10,28,12,100,10,0,1,1.86,False,Middle-Aged
4,5,Engineering,Female,36,Analyst,2021-07-23,3,Bachelor,2,4800.0,...,11,29,13,100,15,9,1,1.25,False,Adult


Once the data has been successfully prepared and the validator is connected to the dataset, the next phase is validation. In this project, I applied seven data expectations to ensure the dataset meets quality and consistency standards.

___
### Expectation 1: To Be Unique

The expectation expect_compound_columns_to_be_unique is used to ensure that the combined values across all columns in a dataset are entirely unique for each row. In other words, each employee has a distinct combination of attributes—akin to a data "fingerprint"—that sets them apart from others.

If you want to include this in your dashboard or validation summary, I can help format it in Markdown or create a concise annotation block. Would you like to add a note about why this matters for downstream analysis, like de-duplication or identity mapping?

In [8]:
# Expectation 1: 
# Ensure the uniqueness of each employee record by validating the combination of 20 key attributes.

validator.expect_compound_columns_to_be_unique(
    column_list = [
    "employee_id", 
    "department", 
    "gender", 
    "age", 
    "job_title", 
    "hire_date",
    "years_at_company", 
    "education_level", 
    "performance_score", 
    "monthly_salary",
    "work_hours_per_week", 
    "projects_handled", 
    "overtime_hours", 
    "sick_days",
    "remote_work_frequency", 
    "team_size", 
    "training_hours", 
    "promotions",
    "employee_satisfaction_score", 
    "resigned",
    "age_cap"])

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 100000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Based on the validation check, none of the combined values across all columns were identical between rows.

- Total rows checked: 100,000

- Unexpected duplicate values: 0

- Missing values: 0

- Unexpected percentage: 0.0%

___
### Expectation 2: To Be Between

The expectation expect_column_values_to_be_between is used to ensure that the values in a given column fall within a specified range.
In this dataset, I applied this expectation to validate whether the monthly salary (Monthly_Salary) of all employees falls within the range of 3,850 to 9,000.

In [9]:
# Expectation 2:
# Ensure that all values in the 'Monthly_Salary' column fall within a reasonable range,
# between 3,850 and 9,000. This range could represent normalized salary units.

validator.expect_column_values_to_be_between(
    column="monthly_salary",
    min_value=3_850,
    max_value=9000
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 100000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Based on the validation results, the monthly salary values (Monthly_Salary) for all employees fall within the expected range of 3,850 to 9,000.

- Total rows checked: 10,000

- Unexpected percentage: 0.0%

- Missing values: 0

- Total unexpected percentage: 0.0%

This confirms that the Monthly_Salary column meets the defined range expectation, with no anomalies or missing values detected in the dataset.

___
### Expectation 3: Most Common Value To Be In Set

The expectation expect_column_most_common_value_to_be_in_set is used to validate whether the most frequent value in a column belongs to a specified set.

In this case, the expectation was applied to determine whether the department with the highest number of employees is either 'IT' or 'Marketing'—effectively verifying which team dominates in terms of headcount.

In [10]:
# Expectation 3:
# Confirm that the most frequently occurring value in the 'Department' column is either 'IT' or 'Marketing'. 
# This helps check for dominant categories. If there is a tie between values, it is still considered valid.

validator.expect_column_most_common_value_to_be_in_set(
    column="department",
    value_set=["IT", "Marketing"],
    ties_okay=True  # set ke True kalau kamu mau toleransi nilai dominan lebih dari 1
)


Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": [
      "Marketing"
    ]
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Based on the validation, the Marketing team has the highest number of employees in the company.

- Most common department (observed value): Marketing

This confirms that the expectation expect_column_most_common_value_to_be_in_set was successfully validated, with the Marketing department being the dominant team among employees.

___
### Expecation 4: Column Values To Be In Set

The expectation expect_column_values_to_be_in_set is used to validate whether the values in a column strictly match a predefined set.
In this case, the expectation was applied to ensure that the educational qualifications of all employees are limited to the following degrees:
- High School
- Bachelor
- Master
- PhD

This validation helps ensure consistency in the dataset and flags any unexpected or misspelled entries that could affect downstream analysis.

In [11]:
# Expectation 4:
# Validate that all values in the 'Education_Level' column match one of the expected options.
# Helps ensure categorical integrity and catch typos or unexpected entries.

validator.expect_column_values_to_be_in_set(
    column="education_level",
    value_set=['High School', 'Bachelor', 'Master', 'PhD']  # Ganti sesuai daftar nilai valid kamu
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 100000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Following the validation, it's confirmed that all employee education levels are limited to the expected values: High School, Bachelor, Master, and PhD.
- Total rows checked: 10,000
- Unexpected percentage: 0.0%
- Missing values: 0
- Total unexpected percentage: 0.0%

This validates the consistency and completeness of the Education_Level column, with no anomalies or missing entries detected.

___
### Expectation 5: Column Values To Not Be Null

The expectation expect_column_values_to_not_be_null is used to verify whether a given column contains any missing values.

In this case, the expectation was applied to check if the 'Resigned' status of all employees is fully populated—ensuring that there are no empty or null values in this column.

In [12]:
# Expectation 5:
# Check that the 'resigned' column has no null (missing) values.
# Essential when this column is used as a target variable or in reporting.

validator.expect_column_values_to_not_be_null(column="resigned")

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 100000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The validation confirms that all employees have their respective 'Resigned' status properly filled.

- Total rows checked: 10,000
- Unexpected percentage: 0.0%

This ensures the completeness of the Resigned column, with no missing or null values detected, maintaining the reliability of resignation-related insights in the dataset.

___
### Expectation 6: Table Column Count To Equal

The expectation expect_table_column_count_to_equal is used to verify whether the dataset contains the correct number of columns as intended.

For this dataset, the expected structure includes 21 data types or columns, representing the full set of employee attributes required for analysis.

In [13]:
# Expectation 6:
# Ensure the dataset has exactly 20 columns as expected by the schema.
# This guards against structural changes in the dataset over time.

validator.expect_table_column_count_to_equal(value=21)

Calculating Metrics:   0%|          | 0/3 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 21
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The validation confirms that the dataset under observation contains exactly 21 columns, aligning with the expected structure for employee data.
- Number of observed columns: 21

This result verifies the integrity of the data schema using the expect_table_column_count_to_equal expectation, ensuring all relevant attributes are present for downstream analysis.


___
### Expectation 7: Column Values To Be Of Type

The expectation expect_column_values_to_be_of_type is used to validate whether the values in a column conform to a specified data type.

In this case, the expectation was applied to verify that the Monthly_Salary column contains values of type float, ensuring numerical precision for downstream calculations and visualizations.

In [14]:
# Expectation 7:
# Validate that values in the 'Monthly_Salary' column are of float data type.
# Important for calculations, aggregations, and preventing type-related bugs.

validator.expect_column_values_to_be_of_type(
    column="monthly_salary",
    type_="float"
)

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The validation confirms that the values in the Monthly_Salary column are indeed of type float64, ensuring compatibility with numerical operations and analytical processes.
- Observed data type: float64

After completing all data validations, the next step is to create a checkpoint, which saves all configured expectations to the dataset. This allows for reproducible validation workflows and serves as a reference for future data quality checks


___

## Checkpoint

Checkpoints are used to:
- Embed data validations into the DAG, enabling seamless integration with pipeline orchestration tools.
- Ensure data quality before further processing, serving as a gatekeeper for clean, reliable inputs.
- Store validation results so they can be visualized or revisited later for audits, debugging, or stakeholder reviews.

In [14]:
# Create a checkpoint
validator.save_expectation_suite(discard_failed_expectations=False)
checkpoint_1 = context.add_or_update_checkpoint(
    name = 'checkpoint_1',
    validator = validator,
)

# Run a checkpoint
checkpoint_result = checkpoint_1.run()

Calculating Metrics:   0%|          | 0/30 [00:00<?, ?it/s]

Next comes the creation of Data Docs, which are auto-generated HTML reports summarizing the data validations executed via checkpoints or expectation suites.

Great Expectations automatically generates visual documentation that includes:
- Expectation status (success/failure)
- Dataset statistics
- Links to the suite and checkpoint
- Metadata and batch details

These Data Docs provide a user-friendly way to audit validation results, share data quality insights, and maintain transparency throughout the pipeline.


___

## Data Docs

In [ ]:
# Build data docs

context.build_data_docs()

With the data validation phase completed using Great Expectations, the next stage involves data visualization through Kibana, leveraging the indexed data in Elasticsearch. This enables interactive exploration and storytelling around employee performance and productivity metrics.

___